In [150]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 
import re
import ast
import gc

In [151]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"

input_directory = Path(f'{main_path}Twitter/Input_test')  
print(input_directory)
max_columns = 8


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/Twitter/Input_test


In [152]:
for i in range(max_columns):
        col_path = [f"col_path_{i}" for i in range(1, max_columns)]
        col_path_values = [f"col_path_{i}_values" for i in range(1, max_columns)]
        col_path_list = [f"col_path_{i}_lIST" for i in range(1, max_columns)]

In [153]:
print(col_path_values)

['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values', 'col_path_6_values', 'col_path_7_values']


### loading_data()

In [154]:
def loading_data(data):
    

    with open(data, 'r') as file:
        data = json.load(file)

    print(type(data))



    df = pd.DataFrame()
    df['col_path_0_values'] = data
    df['json_name'] = df.index
    df['json_name'] = df['json_name'].str.replace(r'^data/', '', regex=True)
    df['col_path_0_LIST'] = ''

   

    col = df.pop('json_name') 
    df.insert(0, 'json_name', col)
    
    df.reset_index(inplace = True)
    df = df.drop('index', axis = 1)

   
    
    for i in range(1,max_columns):   
        df[f'col_path_{i}'] = ''
        df[f'col_path_{i}_values'] = ''
        df[f'col_path_{i}_LIST'] = ''

    index_list = []
    for idx, row in df.iterrows():
        if row['col_path_0_values'] == 'No data':
            index_list.append(idx)
    
    df_no_data = df.loc[index_list]
    df = df.drop(index_list)
   

        

    


    
    return(df, df_no_data)



### flatten_json()

In [155]:
def flatten_json(df):
    
    new_rows = []
    current_rows = df.copy()  # snapshot to iterate over

    for ix, row in current_rows.iterrows():
        value= row['col_path_0_values']
            
        if isinstance(value, list):
                    # Check if list of dicts
            if value and isinstance(value[0], dict):
                list_holder = 'LIST'
                
                seen = set()
                unique = []
                for d in value:
                    # Use json.dumps to get a hashable, sorted representation
                    s = json.dumps(d, sort_keys=True)
                    if s not in seen:
                        seen.add(s)
                        unique.append(d)
                value = unique
                

                for item in value:
                    new_row = row.copy()
                    new_row['col_path_0_values'] = item
                    new_row['col_path_0_LIST'] = list_holder
                    new_rows.append(new_row)
    



    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    new_rows.clear()

           

    new_rows = []

    for i in range(max_columns):
        current_rows = df.copy()  # snapshot to iterate over

        for ix, row in current_rows.iterrows():
            value = row.get(f'col_path_{i}_values', None)
    

            if isinstance(value, dict):
                for k, v in value.items():
                    if isinstance(v, list):
                        # Check if list of dicts
                        if v and isinstance(v[0], dict):

                            seen = set()
                            unique = []
                            for d in v:
                                # Use json.dumps to get a hashable, sorted representation
                                s = json.dumps(d, sort_keys=True)
                                if s not in seen:
                                    seen.add(s)
                                    unique.append(d)
                            v = unique
                            

                            for item in v:
                                new_row = row.copy()
                                new_row[f'col_path_{i+1}_values'] = item
                                new_row[f'col_path_{i+1}'] = k
                                new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                                new_rows.append(new_row)
                        else:
                            # Regular list (e.g., strings, ints)
                            v = v[0]
                            new_row = row.copy()
                            new_row[f'col_path_{i+1}_values'] = v
                            new_row[f'col_path_{i+1}'] = k
                            new_row[f'col_path_{i+1}_LIST'] = 'LIST'
                            new_rows.append(new_row)
                    else:
                
                        new_row = row.copy()
                        new_row[f'col_path_{i+1}_values'] = v
                        new_row[f'col_path_{i+1}'] = k
                        new_row[f'col_path_{i+1}_LIST'] = 'NO LIST'
                        new_rows.append(new_row)
                

        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
        new_rows.clear()
            
                        


    

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df_red = df.drop(columns=[col for col in df.columns if col.endswith("_LIST")])
    
    df_red['last_valid_index'] = df_red.apply(pd.Series.last_valid_index, axis=1)
    df_red = df_red.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    col_path = [f"col_path_{i}" for i in range(1, max_columns)]
    col_path = col_path + ['json_name']
    df = pd.merge(df, df_red, on = col_path, how='left')
    
    
    return(df)

### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [156]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']



# Define the function 'process_col_path()'
def process_col_path(df, col_path_values, data_types):
    df['data_type'] = ''

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """
    
    for ix, row in df.iterrows():
        for col in col_path_values:
            

            #If the value stored in the column is found in the list 'data_types', 
            if row[col] in data_types:
                # this value is placed in the column 'data_type'
                value = row[col]
                
                df.at[ix, 'data_type'] = value
            else:
                continue
               
    
                
    return df

### remove_intermediate_rows()

In [157]:
def remove_intermediate_rows(df):
    drop_index = []
    for ix, row in df.iterrows():
            last_value = row[f"{row['last_valid_index']}"]
           
        
            if isinstance(last_value, str):
                if last_value in (data_types):
                        col =  row['last_valid_index']
                        col_level = re.findall(r'\d+', col)
                        col_level = col_level[0]
                                
                                
                        df.at[ix, f"col_path_{col_level}_LIST"] = np.nan

                                
                else:
                        drop_index.append(ix)
            else:
                drop_index.append(ix)
                  
        
   
    df = df.drop(drop_index)
    df = df.drop(columns=[col for col in df.columns if col.endswith("values")])

    
    return df

### extract_path()

In [158]:
def extract_path(df, max_columns):
    
    df['path']= ''
    
    for ix, row in df.iterrows():
        path = []
    
        for col in [f"col_path_{i}" for i in range(1, max_columns)]:
            val = row[col]
        
            if pd.notna(val):

                path.append(str(val))

        
        df.at[ix, 'path'] = path
       
    
        

    return df




### list_path()

In [159]:
def list_path(df):
    
    df['list_path'] = ''
    df['subfield_path'] = ''
    df['column_name'] = ''
    df['var_type'] = ''
    


    for ix, row in df.iterrows():
        
        path = row['path'] 
        
        
        for i in range( len(path)):
           
            if row[f"col_path_{i}_LIST"] == 'LIST':
                var_type = 'list'
                list_path = json.dumps(path[:-1])
                subfield_path = path[-1]
                column_name = subfield_path
                subfield_path = json.dumps(subfield_path)

            elif row[f"col_path_{i}_LIST"] == "NO LIST":
                var_type = 'static'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
             
            elif row[f"col_path_{i}_LIST"] == 'nan':
                var_type = 'skip'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
               
            else:
                var_type = 'skip'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]


           
    
        df.at[ix, 'list_path'] = list_path
        df.at[ix, 'subfield_path'] = subfield_path
        df.at[ix, 'column_name'] = column_name
        df.at[ix, 'var_type'] = var_type

    
      
        
    return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [160]:
def clean_and_store(df, df_no_data, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """

  
   
    #df=  df[df['col_path_1'] != 'Direct Message']
    
    df = pd.concat([df, df_no_data], ignore_index=True)
    df = df[df['json_name'] != 'manifest.js']
  
    df = df[['json_name','column_name', 'path', 'list_path', 'subfield_path', 'var_type', 'data_type']]
    df = df.astype(str)
    df = df.drop_duplicates()
    # Save the DataFrame 
    df.to_csv(f"{main_path}Twitter/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [161]:
#pd.set_option('display.max_colwidth', 100)

def structure_donations(data, col_path, max_columns):
     # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    df, df_no_data = loading_data(data)
    print('FINISH: loading_data()')
    
   
    df = flatten_json(df)
    print('FINISH: flatten_json()')
   
   
   
    df = process_col_path(df, col_path_values, data_types)
    print('FINISH: process_col_path()')
    df = remove_intermediate_rows(df)
    print('FINISH: remove_intermediate_rows()')
    df = extract_path(df, max_columns)
    print('FINISH: extract_path()')
    
    df = list_path(df)
    print('FINISH: list_path()')
    df = clean_and_store(df, df_no_data, file_name)
    print('FINISH: clean_and_store()')

    del df,df_no_data, data

    
    
    

    

## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [162]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        print(file.name)
        result = structure_donations(file, col_path, max_columns)
        del result
        gc.collect()
        

X_structure_twitter-2024-11-21-326363ec5310e8b3e585b6f4cde18bce0ff75734a36f376d0a2b5b4fc58916da (copy).json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()


/tmp/ipykernel_42970/1142702742.py:97: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


X_structure_twitter-2025-03-15-76701c7591722b08ff3886a6ec1a130366b8ce153f72f649e364ff2f3ecf77bb.json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
X_structure_twitter-2024-11-21-326363ec5310e8b3e585b6f4cde18bce0ff75734a36f376d0a2b5b4fc58916da.json
<class 'dict'>
FINISH: loading_data()


/tmp/ipykernel_42970/1142702742.py:97: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
X_structure_twitter-2025-03-14-27da9bbce1b35bfe71f31166f8e683da63bb589a63202a59ef59c0909e6c2655.json
<class 'dict'>
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()


## Merge all data structures into one schema_df

In [163]:
# Path to the folder containing CSV files
output_path = f"{main_path}Twitter/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()
display(merged_df)


,json_name,column_name,path,list_path,subfield_path,var_type,data_type
0,account-creation-ip.js,accountId,"['accountCreationIp', 'accountId']",NaN,"[""accountCreationIp"", ""accountId""]",static,string
1,account-creation-ip.js,userCreationIp,"['accountCreationIp', 'userCreationIp']",NaN,"[""accountCreationIp"", ""userCreationIp""]",static,string
2,account-timezone.js,accountId,"['accountTimezone', 'accountId']",NaN,"[""accountTimezone"", ""accountId""]",static,string
3,account.js,email,"['account', 'email']",NaN,"[""account"", ""email""]",static,string
4,account.js,createdVia,"['account', 'createdVia']",NaN,"[""account"", ""createdVia""]",static,string
...,...,...,...,...,...,...,...
667,connected-application.js,privacyPolicyUrl,"['connectedApplication', 'organization', 'priv...",NaN,"[""connectedApplication"", ""organization"", ""priv...",static,string
668,connected-application.js,termsAndConditionsUrl,"['connectedApplication', 'organization', 'term...",NaN,"[""connectedApplication"", ""organization"", ""term...",static,string
684,personalization.js,advertisers,"['p13nData', 'interests', 'audienceAndAdvertis...",NaN,"[""p13nData"", ""interests"", ""audienceAndAdvertis...",static,string
818,ip-audit.js,NaN,NaN,NaN,NaN,NaN,NaN


### Id creation

In [164]:

merged_df['name'] = merged_df['json_name'].str.replace(".js", "")

id_df = merged_df[['name', 'path']]
id_df = id_df.drop_duplicates()
id_df['id'] = ''


for n in range(1,max_columns):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = row['path']
        name = row['name']  

        if pd.notna(path):
            
            path =  ast.literal_eval(path)
            print(path)
            #path_zero = path[0]
            path_rest = path[-n:]
            

            

            if not isinstance(path_rest, list):
                path_rest = [path_rest]

            #if [path_zero] == path_rest:
                #id_list = [name] + path_rest
            #else:
                #id_list = [name]+ [path_zero] + path_rest

            id_list = [name] + path_rest

            
            new_id = ':'.join(id_list)

        else:

            new_id = name

        id_df.at[i, 'id'] = new_id  


merged_df = pd.merge(merged_df, id_df, on = ['name', 'path'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 
#merged_df['id']= merged_df['id'].fillna(merged_df['variable'])


['accountCreationIp', 'accountId']
['accountCreationIp', 'userCreationIp']
['accountTimezone', 'accountId']
['account', 'email']
['account', 'createdVia']
['account', 'username']
['account', 'accountId']
['account', 'createdAt']
['account', 'accountDisplayName']
['adsRevenueSharing', 'payoutHistory']
['connectedApplication', 'name']
['connectedApplication', 'description']
['connectedApplication', 'permissions']
['connectedApplication', 'approvedAt']
['connectedApplication', 'id']
['dmConversation', 'conversationId']
['dmConversation', 'conversationId']
['keyRegistryData', 'userId']
['periscopeAccountInformation', 'displayName']
['periscopeAccountInformation', 'digitsId']
['periscopeAccountInformation', 'username']
['periscopeAccountInformation', 'twitterId']
['periscopeAccountInformation', 'id']
['periscopeAccountInformation', 'twitterScreenName']
['periscopeAccountInformation', 'isTwitterUser']
['periscopeAccountInformation', 'createdAt']
['periscopeProfileDescription', 'bio']
['p13nD

#### ID duplicate flags

In [165]:
merged_df = merged_df.drop_duplicates()
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')

/tmp/ipykernel_42970/29815283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup['duplicate_flag'] = 'Yes'


### Clean and Save

In [166]:


keep_columns = ['json_name','id', 'column_name', 
                 'path', 'list_path', 'subfield_path', 
                 'var_type', 'data_type', 'json_name', 'duplicate_flag']


merged_df = merged_df[keep_columns ]

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}Twitter/Final/Merged_structures_TT.csv", index=False)